In [ ]:
import os
import sys
# sys.path.insert(0,'../')
sys.path.insert(0,'/home/zulissi/software/adamwr')
import numpy as np
import cgcnn
#Select which GPU to use if necessary
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES = 1
import mongo


In [ ]:
# import pickle

# docs_all = pickle.load(open('/home/sback/work_dir/new_new_script/CO_final_adsorbate/CO_docs_connectivity.pkl','rb'))['docs']
# SDT_list_relaxed=pickle.load(open('/home/sback/work_dir/new_new_script/CO_final_adsorbate/SDT_list_distance_relaxed.pkl','rb'))['SDT_list_distance_relaxed']
# SDT_list_unrelaxed=pickle.load(open('/home/sback/work_dir/new_new_script/CO_final_adsorbate/SDT_list_distance_unrelaxed.pkl','rb')
# )['SDT_list_distance_unrelaxed']

# docs_all, SDT_list_relaxed, SDT_list_unrelaxed = zip(*[[doc,SDTR, SDTU] for doc,SDTR, SDTU 
#         in zip(docs_all, SDT_list_relaxed, SDT_list_unrelaxed) 
#         if -3.0 < doc['energy'] < 1.0])

# target_list = np.array([doc['energy'] for doc in docs_all]).reshape(-1,1)


In [ ]:
# pickle.dump(docs_all, open('/home/junwoony/Desktop/cgcnn/CO_20000_docs.pkl', 'wb'))
# pickle.dump(SDT_list_unrelaxed, open('/home/junwoony/Desktop/cgcnn/CO_20000_SDT_list_unrelaxed.pkl', 'wb'))
# pickle.dump(SDT_list_relaxed, open('/home/junwoony/Desktop/cgcnn/CO_20000_SDT_list_relaxed.pkl', 'wb'))
# pickle.dump(target_list, open('/home/junwoony/Desktop/cgcnn/CO_20000_target_list.pkl', 'wb'))



In [ ]:
# import pickle

# docs_all = pickle.load(open('/home/junwoony/Desktop/cgcnn/CO_20000_docs.pkl', 'rb'))
# SDT_list_unrelaxed = pickle.load(open('/home/junwoony/Desktop/cgcnn/CO_20000_SDT_list_unrelaxed.pkl', 'rb'))
# structures = SDT_list_unrelaxed[0]
# # SDT_list_relaxed = pickle.load(open('/home/junwoony/Desktop/cgcnn/CO_20000_SDT_list_relaxed.pkl', 'rb'))
# # structures = SDT_list_relaxed[0]

# target_list = pickle.load(open('/home/junwoony/Desktop/cgcnn/CO_20000_target_list.pkl', 'rb'))

# #Settings necessary to build the model (since they are size of vectors as inputs)
# orig_atom_fea_len = structures[0].shape[-1]
# nbr_fea_len = structures[1].shape[-1]


In [ ]:
# from torch.utils.data import Dataset, DataLoader
# import mongo
# from cgcnn.data_JY import StructureData, ListDataset, StructureDataTransformer
# import numpy as np
# import tqdm
# from sklearn.preprocessing import StandardScaler


# SDT = StructureDataTransformer(atom_init_loc='/home/zulissi/software/cgcnn_sklearn/atom_init.json',
#                               max_num_nbr=12,
#                                step=0.2,
#                               radius=1,
#                               use_tag=False,
#                               use_fixed_info=False,
#                               use_distance=True,
#                               train_geometry = 'final-adsorbate'
#                               )

# SDT_out = SDT.transform(docs_all)

# structures = SDT_out[0]

# #Settings necessary to build the model (since they are size of vectors as inputs)
# orig_atom_fea_len = structures[0].shape[-1]
# nbr_fea_len = structures[1].shape[-1]

# import multiprocess as mp
# from sklearn.model_selection import ShuffleSplit

# SDT_out = SDT.transform(docs_all)

# with mp.Pool(4) as pool:
#     SDT_list_unrelaxed = list(tqdm.tqdm(pool.imap(lambda x: SDT_out[x],range(len(SDT_out)),chunksize=40),total=len(SDT_out)))

In [ ]:
from torch.optim import Adam, SGD
from sklearn.model_selection import ShuffleSplit
from skorch.callbacks import Checkpoint, LoadInitState #needs skorch 0.4.0, conda-forge version at 0.3.0 doesn't cut it

# from cgcnn.data import collate_pool, MergeDataset
# from cgcnn.model import CrystalGraphConvNet

from cgcnn.data_JY import collate_pool, MergeDataset
from cgcnn.model_JY import CrystalGraphConvNet
from skorch import NeuralNetRegressor
import torch
import skorch.callbacks.base


cuda = torch.cuda.is_available()
if cuda:
    device = torch.device("cuda")
else:
    device='cpu'

#Make a checkpoint to save parameters every time there is a new best for validation lost
cp = Checkpoint(monitor='valid_loss_best',fn_prefix='valid_best_')

#Callback to load the checkpoint with the best validation loss at the end of training

class train_end_load_best_valid_loss(skorch.callbacks.base.Callback):
    def on_train_end(self, net, X, y):
        net.load_params('valid_best_params.pt')
        
load_best_valid_loss = train_end_load_best_valid_loss()

In [ ]:
# from sklearn.model_selection import train_test_split

# indices = np.arange(len(SDT_list_unrelaxed))
# SDT_training, SDT_test, target_training, target_test, train_idx, test_idx \
# = train_test_split(SDT_list_unrelaxed, target_list, indices, test_size=0.2)

In [ ]:
# import pickle
# pickle.dump(docs_all, open('/home/junwoony/Desktop/cgcnn/cgcnn_vis/CO_20000_docs.pkl', 'wb'))
# pickle.dump(SDT_list_unrelaxed, open('/home/junwoony/Desktop/cgcnn/cgcnn_vis/CO_20000_SDT_list_unrelaxed.pkl', 'wb'))
# pickle.dump(target_list, open('/home/junwoony/Desktop/cgcnn/cgcnn_vis/CO_20000_target_list_unrelaxed.pkl', 'wb'))

# pickle.dump(SDT_training, open('/home/junwoony/Desktop/cgcnn/cgcnn_vis/CO_20000_SDT_training.pkl', 'wb'))
# pickle.dump(SDT_test, open('/home/junwoony/Desktop/cgcnn/cgcnn_vis/CO_20000_SDT_test.pkl', 'wb'))
# pickle.dump(target_training, open('/home/junwoony/Desktop/cgcnn/cgcnn_vis/CO_20000_target_training.pkl', 'wb'))
# pickle.dump(target_test, open('/home/junwoony/Desktop/cgcnn/cgcnn_vis/CO_20000_target_test.pkl', 'wb'))
# pickle.dump(train_idx, open('/home/junwoony/Desktop/cgcnn/cgcnn_vis/CO_20000_train_idx.pkl', 'wb'))
# pickle.dump(test_idx, open('/home/junwoony/Desktop/cgcnn/cgcnn_vis/CO_20000_test_idx.pkl', 'wb'))


In [ ]:
import pickle 
docs = pickle.load(open('/home/junwoony/Desktop/cgcnn/cgcnn_vis/CO_20000_docs.pkl', 'rb'))
SDT_list_unrelaxed = pickle.load(open('/home/junwoony/Desktop/cgcnn/cgcnn_vis/CO_20000_SDT_list_unrelaxed.pkl', 'rb'))
target_list = pickle.load(open('/home/junwoony/Desktop/cgcnn/cgcnn_vis/CO_20000_target_list_unrelaxed.pkl', 'rb'))

SDT_training = pickle.load(open('/home/junwoony/Desktop/cgcnn/cgcnn_vis/CO_20000_SDT_training.pkl', 'rb'))
SDT_test = pickle.load(open('/home/junwoony/Desktop/cgcnn/cgcnn_vis/CO_20000_SDT_test.pkl', 'rb'))
target_training = pickle.load(open('/home/junwoony/Desktop/cgcnn/cgcnn_vis/CO_20000_target_training.pkl', 'rb'))
target_test = pickle.load(open('/home/junwoony/Desktop/cgcnn/cgcnn_vis/CO_20000_target_test.pkl', 'rb'))
train_idx = pickle.load(open('/home/junwoony/Desktop/cgcnn/cgcnn_vis/CO_20000_train_idx.pkl', 'rb'))
test_idx = pickle.load(open('/home/junwoony/Desktop/cgcnn/cgcnn_vis/CO_20000_test_idx.pkl', 'rb'))

structures = SDT_training[0]
orig_atom_fea_len = structures[0].shape[-1]
nbr_fea_len = structures[1].shape[-1]

In [ ]:
from skorch.dataset import CVSplit
from skorch.callbacks.lr_scheduler import WarmRestartLR, LRScheduler
from adamw import AdamW
from cosine_scheduler import CosineLRWithRestarts
from sklearn.model_selection import train_test_split

train_test_splitter = ShuffleSplit(test_size=0.25, random_state=42)

# warm restart scheduling from https://arxiv.org/pdf/1711.05101.pdf
LR_schedule = LRScheduler(CosineLRWithRestarts, batch_size=214, epoch_size=len(SDT_training), restart_period=10, t_mult=1.2)

#############
# To extract intermediate features, set the forward takes only the first return value to calculate loss
class MyNet(NeuralNetRegressor):
    def get_loss(self, y_pred, y_true, **kwargs):
        y_pred = y_pred[0] if isinstance(y_pred, tuple) else y_pred  # discard the 2nd output
        return super().get_loss(y_pred, y_true, **kwargs)
## return features = net.forward(SDT_test)
############

net = MyNet(
    CrystalGraphConvNet,
    module__orig_atom_fea_len = orig_atom_fea_len,
    module__nbr_fea_len = nbr_fea_len,
    batch_size=214,
    module__classification=False,
    lr=0.0056,
    max_epochs= 200, 
    module__atom_fea_len=46,
    module__h_fea_len=83,
    module__n_conv=6, #8
    module__n_h=2,
    optimizer__weight_decay=1e-5,
    optimizer=AdamW,
    iterator_train__pin_memory=True,
    iterator_train__num_workers=0,
    iterator_train__collate_fn = collate_pool,
    iterator_valid__pin_memory=True,
    iterator_valid__num_workers=0,
    iterator_valid__collate_fn = collate_pool,
    device=device,
#     criterion=torch.nn.MSELoss,
    criterion=torch.nn.L1Loss,
    dataset=MergeDataset,
    train_split = CVSplit(cv=train_test_splitter),
    callbacks=[cp, load_best_valid_loss, LR_schedule]
)

In [ ]:
net.initialize()
net.fit(SDT_training, target_training)

In [ ]:
# net.initialize()
# net.load_params(f_history='valid_best_history.json',
#                 f_optimizer= 'valid_best_optimizer.pt', 
#                 f_params='valid_best_params.pt'
#                 )

In [ ]:
import pandas as pd

train_indices, valid_indices = next(train_test_splitter.split(SDT_training))

training_data = {'actual_value':target_training[train_indices].reshape(-1),
                 'predicted_value':net.predict(SDT_training)[train_indices].reshape(-1)}
validation_data = {'actual_value':target_training[valid_indices].reshape(-1),
                 'predicted_value':net.predict(SDT_training)[valid_indices].reshape(-1)}
test_data = {'actual_value':target_test.reshape(-1),
                 'predicted_value':net.predict(SDT_test).reshape(-1)}

df_training = pd.DataFrame(training_data)
df_validation = pd.DataFrame(validation_data)
df_test = pd.DataFrame(test_data)


In [ ]:
# df_training.to_csv('training.csv', sep='\t', index=True)
# df_test.to_csv('test.csv', sep='\t', index=True)
# df_validation.to_csv('validation.csv', sep='\t', index=True)

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

f, ax = plt.subplots(figsize=(8,8))

ax.scatter(df_training['actual_value'], df_training['predicted_value'], color='orange', 
           marker='o', alpha=0.5, label='train\nMAE=%0.2f, RMSE=%0.2f, R$^2$=%0.2f'\
            %(mean_absolute_error(df_training['actual_value'], df_training['predicted_value']), 
              np.sqrt(mean_squared_error(df_training['actual_value'], df_training['predicted_value'])),
              r2_score(df_training['actual_value'], df_training['predicted_value'])))


ax.scatter(df_validation['actual_value'], df_validation['predicted_value'], color='blue', 
           marker='o', alpha=0.5, label='valid\nMAE=%0.2f, RMSE=%0.2f, R$^2$=%0.2f'\
            %(mean_absolute_error(df_validation['actual_value'], df_validation['predicted_value']), 
              np.sqrt(mean_squared_error(df_validation['actual_value'], df_validation['predicted_value'])),
              r2_score(df_validation['actual_value'], df_validation['predicted_value'])))

ax.scatter(df_test['actual_value'], df_test['predicted_value'], color='green', 
           marker='o', alpha=0.5, label='test\nMAE=%0.2f, RMSE=%0.2f, R$^2$=%0.2f'\
            %(mean_absolute_error(df_test['actual_value'], df_test['predicted_value']), 
              np.sqrt(mean_squared_error(df_test['actual_value'], df_test['predicted_value'])),
              r2_score(df_test['actual_value'], df_test['predicted_value'])))


ax.plot([min(df_training['actual_value']), max(df_training['actual_value'])], 
        [min(df_training['actual_value']), max(df_training['actual_value'])], 'k--')

# format graph
ax.tick_params(labelsize=14)
ax.set_xlabel('DFT E (eV)', fontsize=14)
ax.set_ylabel('CGCNN predicted E (eV)', fontsize=14)
ax.set_title('Multi-element ', fontsize=14) 
ax.legend(fontsize=12)
plt.savefig('prediction.png')
plt.show()


In [ ]:
#Get indices of accurate predictions

# visual_idx = np.where(abs(test_data['actual_value'] - test_data['predicted_value']) < 0.0001)[0]
visual_idx = np.where(abs(training_data['actual_value'] - training_data['predicted_value']) < 0.0001)[0]

for idx in visual_idx:
    out, atom_fea_vis, atom_fea = net.forward([SDT_training[idx]])
    contributions = atom_fea_vis.cpu().data.numpy().reshape(-1)
    # Get idx from docs
    doc_test_idx = train_idx[idx]
    atoms = mongo.make_atoms_from_doc(docs[doc_test_idx])
    atoms.set_initial_charges(contributions)
    atoms.write('./traj/%d.traj'%(idx))


In [ ]:
print('atom feature before considering connectivity \n',atom_fea.reshape(-1),'\n')
print('atom feature after considering connectivity \n', atom_fea_vis.reshape(-1),'\n')
print('contributions of atoms \n', contributions,'\n')
print('mean of the contributions \n', torch.sum(atom_fea_vis.reshape(-1))/len(torch.nonzero(atom_fea_vis.reshape(-1))),'\n')
print('prediction (must be equal to the mean above) \n', out)